In [ ]:
# Importaciones de las librerias necesarias
#%pip install xlrd
#%pip install pandas
#%pip install openpyxl

import pandas as pd

file_path_Datos = r'..\raw_data\Perfil_EmpresasImpo_2022_WEB.xlsx'

file_path = r'raw_data'

columnas_a_leer = ['Cod_Provincia', 'Provincia_Ubicacion', 'Valor_FOB_USD', 'Valor_CIF_USD', 'Capitulo_desc_SA']

#Creación del dataframe con todos los datos

df = pd.read_excel(file_path_Datos, usecols=columnas_a_leer)

#Creación de dataframe sumando todos los costos de importación por provincia 
df_grouped = df.groupby('Provincia_Ubicacion')['Valor_CIF_USD'].sum().reset_index()


# Ahora 'Valor_CIF_USD' es una columna en 'df_grouped' y podemos aplicar el formato
df_grouped['Valor_CIF_USD'] = df_grouped['Valor_CIF_USD'].apply(lambda x: '{:.2f}'.format(x))

print(df_grouped)

In [ ]:
df_grouped['Valor_CIF_USD'] = pd.to_numeric(df_grouped['Valor_CIF_USD'], errors='coerce')

# Calcula la sumatoria total del costo de importaciones
total_costo_importaciones = df_grouped['Valor_CIF_USD'].sum()

print(total_costo_importaciones)
#dieciocho mil quinientos cuarenta y dos millones setenta y tres mil seiscientos setenta y siete con ochenta y cinco centavos.

# Calcula el porcentaje de costo de importación por provincia
df_grouped['Porcentaje'] = (df_grouped['Valor_CIF_USD'] / total_costo_importaciones) * 100

df_grouped_sorted = df_grouped.sort_values('Porcentaje', ascending=False)

# Imprime el DataFrame para verificar los resultados
df_grouped[['Provincia_Ubicacion', 'Valor_CIF_USD', 'Porcentaje']]





In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))  # Tamaño del gráfico (opcional)

# Crear el gráfico de barras con los datos ordenados
plt.bar(df_grouped_sorted['Provincia_Ubicacion'], df_grouped_sorted['Porcentaje'], width=0.6)

# Rotar los nombres de las etiquetas del eje X a vertical
plt.xticks(rotation=90)

# Personalizar el gráfico (opcional)
plt.ylabel('Porcentaje')
plt.xlabel('Provincia')
plt.title('Porcentaje de Valor CIF por Provincia (Ordenado)')

# for index, row in df_grouped_sorted.iterrows():
#     plt.text(index, row['Porcentaje'], f"{row['Porcentaje']:.2f}%", ha='center', va='bottom')

# Mostrar el gráfico
plt.tight_layout()  # Ajusta automáticamente los parámetros del subplot para que el gráfico se ajuste al área de dibujo
plt.show()

In [ ]:
from folium.features import GeoJsonTooltip
import geopandas as gpd
import folium.plugins
import pandas as pd
import folium as f
import json

def draw_map(tls, data_geojson, cols, aliases, df, lefton, lyrname, keyon, fcolor, lgname, nfcolor, higlt):
    center = [18.483402,-69.929611]
    mapa = f.Map(location=center, zoom_start=8,
                min_zoom=1, max_bounds=True,
                min_lat=-84, min_lon=-175,
                max_lat=84,  max_lon=187,
                control_scale=True,
                tiles=tls # cartodb positron
    )

    # Assuming that data_geojson is a dictionary containing the GeoJSON data
    gpd_geo = gpd.GeoDataFrame.from_features(data_geojson)

    lcol, rcol, mcol = cols
    xalias, yalias, zalias = aliases
    
    # Merge the GeoJSON data with df
    gpd_geo = gpd_geo.merge(df, left_on=lefton, right_on=lcol)

    # Create a Choropleth layer with the merged data
    choropleth = f.Choropleth(
        geo_data=gpd_geo.__geo_interface__,
        name = lyrname,
        data=df,
        columns=[lcol, rcol, mcol],
        key_on=keyon,
        fill_color=fcolor, 
        fill_opacity=0.7, 
        line_opacity=0.1,
        legend_name=lgname,
        nan_fill_color=nfcolor,
        line_color = "#0000",
        reset=True,
        highlight=higlt,
    ).add_to(mapa)

    # Add tooltips to the Choropleth layer
    choropleth.geojson.add_child(
        GeoJsonTooltip(fields=[lcol, rcol, mcol],
                       aliases=[f'{xalias}: ', f'{yalias}: ', f'{zalias}: '],
                       labels=True,
                       localize=True,
                       sticky=True)
    )

    f.plugins.Fullscreen(
    position="topright",
    title="Expand me",
    title_cancel="Exit me",
    force_separate_button=True,
    ).add_to(mapa)
        
    f.LayerControl().add_to(mapa)
    
    return mapa

def open_json(path):
    return json.load(open(path))

In [ ]:
# df_grouped[['Provincia_Ubicacion', 'Valor_CIF_USD', 'Porcentaje']]

geojson_path = r'../maps/dominican_republic/geoBoundaries-DOM-ADM1-all-PROVINCES/geoBoundaries-DOM-ADM1.geojson'

options = {
            "tls": 'openstreetmap' , # cartodb positron
            "data_geojson": open_json(geojson_path), 
            "cols": ['Provincia_Ubicacion', 'Valor_CIF_USD', 'Porcentaje'],
            "aliases": ['Provincia', 'Costo', 'Porcentaje'],
            "df": df_grouped,
            "lefton": 'shapeName',
            "lyrname": 'Importaciones',
            "keyon": 'properties.shapeName',
            "fcolor": 'Pastel2',
            "lgname": 'Provincias e importaciones (2022)',
            "nfcolor": 'gray',
            "higlt": True,
          }

imap = draw_map(**options)
imap

In [ ]:
def save_map(m, output_path):
    m.save(f"{output_path}\map.html")
    
save_map(imap, 'output')